# Goal

Show that for a given neutrino MCMC, with some fiducial cosmology, that we can reproduce the same results using a Fisher forecast for an equivalent cosmology

# Logistics

None

# Setup Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment (we use Python 3.6.8) there if you'd like: 
```
$ cd .../<project-directory>
$ python -m virtualenv env 
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

6) Create a data folder where the analysis can store spectrum data. This can be anywhere you'd like - you'll specify the path below. 
```
$ mkdir <project-directory>/data
```

7) Install and build CLASS (if you don't already have a build). Note the `cosmicfish` package includes a method for downloading and installing CLASS for you:
```
$ python 
>>> import cosmicfish as cf
>>> cf.install_class('<project-directory>/class')
>>> exit()
```

# Import & Configure _cosmicfish_

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

Specify the paths from the setup of your analysis environment above.  

In [ ]:
#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/datastore/")
classpath = os.path.join(projectdir, "class")

Specify the granularity of numerical derivatives.

In [ ]:
derivative_step = 0.01 #How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 #For calculating numerical integral wrt mu between -1 and 1 

# Specify Fiducial Cosmologies

In [ ]:
neutrinofid = {
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 0.03, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : None #Not used for neutrino forecasts
        } 

# Specify Experiment Observational Parameters

In [ ]:
#Specify redshift bins, noise per bin, and sky coverage
z_table = np.arange(0.65, 1.85, 0.1)
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `convergence` class of `cosmicfish`. 

We pass to `convergence` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology by step sizes specified by the user and compute the corresponding power spectrum derivatives. 

All 'varyfactors' are computed relative to the fiducial cosmology. That is:

dtheta = varyfactor * theta_fiducial

**WARNING: This process takes a considerable amount of time (~5 mins per varyfactor w/o prexisting data). Only run when you need to.**

In [ ]:
neutrinoconvergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
    neutrinofid, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    mu_step=mu_integral_step,
    varyfactors=[0.003, 0.004, 0.005, 0.006, 0.007] # Relative factors used to compute convergence
    )
neutrinoconvergencetest.gen_all_plots() # Display convergence plots

# Run Forecasts

In [ ]:
neutrinoforecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    neutrinofid, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

neutrinoforecast.gen_pm()

neutrinoforecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                    
        #'omega_ncdm',                                 
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)

# Add CMB Data

In [ ]:
neutrinoforecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'M_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

# Save Fisher Matrices

In [ ]:
neutrinoforecast.export_matrices("~/Desktop")

# Plot Fisher Ellipses on MCMC Contour

In the last step, we exported three Fisher matrices. The next steps are performed using MontePython outside this notebook:

```
$ mv ~/Desktop/inv_fullfisher.dat <project-directory>/chains/inv_fisher.dat
$ cd <project-directory>
$ ./montepython_public/montepython/MontePython.py info chain/ --plot-fisher --center-fisher
```

Then, the desired triangle plot is located in <project-directory>/chains/plots/triangle.pdf

In [ ]:
neutrinoforecast.psterms

In [ ]:
import dill
dill.dump_session('/Users/nicholasdeporzio/Desktop/MCMC_Match.db')

In [ ]:
dill.load_session('notebook_env.db')